# Ideal tables

In [4]:
# import libraries and data

import pandas as pd
import numpy as np
from os import path
from functools import reduce
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from itertools import combinations

file_dir = "data/cleaned/"
file_dir_consolidated = "data/consolidated/"

In [12]:
def compare_tables_drop_duplicates(df1, df2, common_columns_df1, common_columns_df2):
    '''
    Compare two tables based on specified columns and drop duplicates.

    Parameters:
    - df1 (pandas.DataFrame): The first DataFrame.
    - df2 (pandas.DataFrame): The second DataFrame.
    - common_columns_df1 (list): Columns used in df1 to find common rows.
    - common_columns_df2 (list): Columns used in df2 to find common rows.

    Returns:
    - common_rows (pandas.DataFrame): Rows common to both DataFrames with duplicates dropped.
    - unique_rows_df1 (pandas.DataFrame): Rows unique to df1 with duplicates dropped.
    - unique_rows_df2 (pandas.DataFrame): Rows unique to df2 with duplicates dropped.

    Example:
    >>> df1 = pd.DataFrame({'Company': ['A', 'B', 'C'], 'Project name': ['P1', 'P2', 'P3'], 'Country': ['X', 'Y', 'Z'], 'Year': [2020, 2021, 2022]})
    >>> df2 = pd.DataFrame({'Full company name': ['A Corp', 'B Corp', 'D Corp'], 'Company type': ['Type1', 'Type2', 'Type3'], 'Company ID number': [101, 102, 103], 'Country': ['X', 'Y', 'Z'], 'Year': [2020, 2021, 2023]})
    >>> common_cols_df1 = ['Country', 'Year']
    >>> common_cols_df2 = ['Country', 'Year']
    >>> common, unique_df1, unique_df2 = compare_tables_drop_duplicates(df1, df2, common_cols_df1, common_cols_df2)
    >>> print(common)
      Country  Year
    0       X  2020
    1       Y  2021
    >>> print(unique_df1)
      Company Project name
    2       C           P3
    >>> print(unique_df2)
      Full company name Company type  Company ID number
    2            D Corp       Type3                103
    '''

    # Find common rows
    common_rows = pd.merge(df1, df2, left_on=common_columns_df1, right_on=common_columns_df2, how='inner')

    # Drop duplicates in common rows
    common_rows = common_rows.drop_duplicates(subset=common_columns_df1)

    # Drop duplicates in unique rows in df1
    unique_rows_df1 = df1[~df1.set_index(common_columns_df1).index.isin(common_rows.set_index(common_columns_df1).index)]
    unique_rows_df1 = unique_rows_df1.drop_duplicates(subset=common_columns_df1)

    # Drop duplicates in unique rows in df2
    unique_rows_df2 = df2[~df2.set_index(common_columns_df2).index.isin(common_rows.set_index(common_columns_df2).index)]
    unique_rows_df2 = unique_rows_df2.drop_duplicates(subset=common_columns_df2)

    return {"in table 1 but not in table 2": unique_rows_df1, 
            "in table 2 but not in table 1": unique_rows_df2,
            "in both tables": common_rows}


## Ideal Companies (Part 3A)

From the complete companies cleaned data:
1. Per report (country, year), create a pivot table based on Company name
2. Combine the pivot tables into a list
3. The result is a list of companies per report (w/ revenue value or payments to governments report summarized)

From this data, we can get:
1. a consistent list of companies per report (country, year) in relation to Part 5
2. a list of all companies that has ever appeared in a country report

In [ ]:
companies_5_complete_cleaned = pd.read_csv(path.join(file_dir, "companies-5-complete-cleaned.csv"))

companies_grouped_data = companies_5_complete_cleaned.groupby(['Country', 'Year'])

companies_pivot_tables = []

# Iterate over each group and create a pivot table based on Company
for (country, year), group_df in companies_grouped_data:
    companies_pivot_table = group_df.pivot_table(index='Company', aggfunc={"Revenue value": "sum"}) 
    companies_pivot_table['Country'] = country
    companies_pivot_table['Year'] = year
    companies_pivot_tables.append(companies_pivot_table)

companies_result_df = pd.concat(companies_pivot_tables).reset_index()

companies_result_df = companies_result_df.rename(columns={"Company": "Full company name", "Revenue value": "Payments to Governments Report"})

# companies_result_df.to_csv("data/cleaned/companies_3a_ideal.csv", index=False)

companies_3a_ideal = companies_result_df.copy()

display(companies_3a_ideal)

### Additional columns

In [71]:
companies_3a_actual = pd.read_csv(path.join(file_dir_consolidated, "Part 3 - Reporting companies' list.csv"))
companies_3a_actual_allcaps = companies_3a_actual.copy()
companies_3a_actual_allcaps["Full company name"] = companies_3a_actual_allcaps["Full company name"].str.upper()
# companies_3a_actual_allcaps["Government entity"] = companies_5_actual_allcaps["Government entity"].str.upper()

common_columns_3a = ["Full company name", "Country", "Year"]

# display(companies_3a_actual_allcaps)

companies_merged = pd.merge(companies_3a_ideal, companies_3a_actual_allcaps, on=['Full company name', 'Country', 'Year'], how='left')

# companies_merged.to_csv("data/cleaned/companies_5_.csv", index=False)

display(companies_merged)
companies_merged.to_csv("data/cleaned/companies_3a_merge_full.csv", index=False)

,Full company name,Payments to Governments Report_x,Country,Year,Company type,Company ID number,Sector,Commodities (comma-seperated),Stock exchange listing or company website,"Audited financial statement (or balance sheet, cash flows, profit/loss statement if unavailable)",Payments to Governments Report_y,ISO Code,Start Date,End Date
0,ABAAN RAYAN LIMITED,1.025688e+07,Afghanistan,2018,Private,9004655032,Other,Coal,NaN,Not available,10256884,AFG,2017-12-21,2018-12-22
1,ABBAS GHAZNAVI LIMITED,3.624451e+07,Afghanistan,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABDUL FATAH,1.394220e+05,Afghanistan,2018,Private,9001846329,Other,Construction stone,Not applicable,Not available,139422,AFG,2017-12-21,2018-12-24
3,ABDUL RAOUF,5.092500e+04,Afghanistan,2018,Private,9002825819,Other,Construction stone,Not applicable,Not available,50925,AFG,2017-12-21,2018-12-24
4,ABDUL WAHAB ARIOBWAL LIMITED,4.588710e+05,Afghanistan,2018,Private,9004775335,Other,Talc,NaN,Not available,458871,AFG,2017-12-21,2018-12-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3852,MAAMBA COLLIERIES LIMITED,4.703749e+08,Zambia,2019,Private,1001594184,Mining,"COA, PYR",http://www.maambacoal.com/,NaN,470374861.13,ZMB,2019-01-01,2019-12-31
3853,MOPANI COPPER MINES PLC,5.143412e+08,Zambia,2019,Private,1001630233,Mining,Cu,NaN,NaN,514341227.78,ZMB,2019-01-01,2019-12-31
3854,NFC AFRICA MINING PLC,2.559570e+08,Zambia,2019,Private,NaN,Mining,Au,http://www.cnmc.com.cn/indexen.jsp,NaN,255956974.17,ZMB,2019-01-01,2019-12-31
3855,SINO METALS,1.007475e+08,Zambia,2019,Private,NaN,Mining,NaN,NaN,NaN,100747457.08,ZMB,2019-01-01,2019-12-31


In [72]:
# Group ideal by Country and Year
ideal_groups = companies_3a_ideal.groupby(['Country', 'Year'])

# Initialize an empty list to store merged DataFrames
companies_merged_list = []

# companies_3a_actual_allcaps['Country'] 

# Iterate over each group and merge with actual
for (country, year), group_ideal in ideal_groups:
    # display(group_ideal)
    group_actual = companies_3a_actual_allcaps[(companies_3a_actual_allcaps['Country'] == country) & (companies_3a_actual_allcaps['Year'] == year)]
    # display(group_actual)
    companies_merged_tolist = pd.merge(group_ideal, group_actual, on=['Full company name', 'Country', 'Year'], how='left')
    # display(companies_merged_tolist)
    companies_merged_list.append(companies_merged_tolist)
    # display(companies_merged_list)

# Concatenate the list of merged DataFrames into a single DataFrame
companies_final_merged = pd.concat(companies_merged_list)

display(companies_final_merged)
companies_final_merged.to_csv("data/cleaned/companies_3a_merge_per_report.csv", index=False)

,Full company name,Payments to Governments Report_x,Country,Year,Company type,Company ID number,Sector,Commodities (comma-seperated),Stock exchange listing or company website,"Audited financial statement (or balance sheet, cash flows, profit/loss statement if unavailable)",Payments to Governments Report_y,ISO Code,Start Date,End Date
0,ABAAN RAYAN LIMITED,1.025688e+07,Afghanistan,2018,Private,9004655032,Other,Coal,NaN,Not available,10256884,AFG,2017-12-21,2018-12-22
1,ABBAS GHAZNAVI LIMITED,3.624451e+07,Afghanistan,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABDUL FATAH,1.394220e+05,Afghanistan,2018,Private,9001846329,Other,Construction stone,Not applicable,Not available,139422,AFG,2017-12-21,2018-12-24
3,ABDUL RAOUF,5.092500e+04,Afghanistan,2018,Private,9002825819,Other,Construction stone,Not applicable,Not available,50925,AFG,2017-12-21,2018-12-24
4,ABDUL WAHAB ARIOBWAL LIMITED,4.588710e+05,Afghanistan,2018,Private,9004775335,Other,Talc,NaN,Not available,458871,AFG,2017-12-21,2018-12-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,MAAMBA COLLIERIES LIMITED,4.703749e+08,Zambia,2019,Private,1001594184,Mining,"COA, PYR",http://www.maambacoal.com/,NaN,470374861.13,ZMB,2019-01-01,2019-12-31
12,MOPANI COPPER MINES PLC,5.143412e+08,Zambia,2019,Private,1001630233,Mining,Cu,NaN,NaN,514341227.78,ZMB,2019-01-01,2019-12-31
13,NFC AFRICA MINING PLC,2.559570e+08,Zambia,2019,Private,NaN,Mining,Au,http://www.cnmc.com.cn/indexen.jsp,NaN,255956974.17,ZMB,2019-01-01,2019-12-31
14,SINO METALS,1.007475e+08,Zambia,2019,Private,NaN,Mining,NaN,NaN,NaN,100747457.08,ZMB,2019-01-01,2019-12-31


In [70]:
print("Duplicate rows removed")
for key, data in compare_tables_drop_duplicates(companies_3a_ideal, companies_final_merged, common_columns_3a, common_columns_3a).items():
    print(f'{key}: {data.shape[0]} rows')

Duplicate rows removed
in table 1 but not in table 2: 0 rows
in table 2 but not in table 1: 0 rows
in both tables: 3853 rows


## Ideal Agencies (Part 3B)

From the complete agencies cleaned data:
1. Per report (country, year), create a pivot table based on Agency name
2. Combine the pivot tables into a list
3. The result is a list of government agencies per report (w/ revenue value or total reported summarized)

From this data, we can get:
1. a consistent list of agencies per report (country, year) in relation to Part 5
2. a list of all agencies that has ever appeared in a country report

In [73]:
agencies_4_complete_indexed_cleaned = pd.read_csv(path.join(file_dir, "agencies-4-actual-complete-indexed-cleaned.csv"))

agencies_grouped_data = agencies_4_complete_indexed_cleaned.groupby(['Country', 'Year'])

agencies_pivot_tables = []

# Iterate over each group and create a pivot table based on Company
for (country, year), group_df in agencies_grouped_data:
    agencies_pivot_table = group_df.pivot_table(index='Government entity', aggfunc={"Revenue value": "sum"}) 
    agencies_pivot_table['Country'] = country
    agencies_pivot_table['Year'] = year
    agencies_pivot_tables.append(agencies_pivot_table)

agencies_result_df = pd.concat(agencies_pivot_tables).reset_index()

agencies_result_df = agencies_result_df.rename(columns={"Government entity": "Full name of agency", "Revenue value": "Total reported"})

agencies_result_df.to_csv("data/cleaned/agencies_3b_ideal.csv", index=False)

agencies_3b_ideal = agencies_result_df.copy()

display(agencies_3b_ideal)

,Full name of agency,Total reported,Country,Year
0,MINISTRY OF FINANCE (CUSTOMS DEPARTMENT),1.459912e+09,Afghanistan,2018
1,MINISTRY OF FINANCE (REVENUE DEPARTMENT),1.089699e+09,Afghanistan,2018
2,MINISTRY OF INDUSTRY AND COMMERCE,0.000000e+00,Afghanistan,2018
3,MINISTRY OF MINES AND PETROLEUM (REVENUE DEPAR...,2.285251e+09,Afghanistan,2018
4,NATIONAL ENVIRONMENTAL PROTECTION AGENCY,4.700000e+04,Afghanistan,2018
...,...,...,...,...
537,MINISTRY OF LANDS,9.827895e+05,Zambia,2019
538,MINISTRY OF MINES AND MINERALS DEVELOPMENT,4.128300e+07,Zambia,2019
539,MINISTRY OF MINES AND MINERALS DEVELOPMENT - P...,3.051347e+06,Zambia,2019
540,ZAMBIA CONSOLIDATED COPPER MINES – INVESTMENT ...,0.000000e+00,Zambia,2019


### Additional columns

In [80]:
agencies_3b_actual = pd.read_csv(path.join(file_dir_consolidated, "Part 3 - Reporting government entities list.csv"))
agencies_3b_actual_allcaps = agencies_3b_actual.copy()
agencies_3b_actual_allcaps["Full name of agency"] = agencies_3b_actual_allcaps["Full name of agency"].str.upper()
# companies_3a_actual_allcaps["Government entity"] = companies_5_actual_allcaps["Government entity"].str.upper()

common_columns_3b = ["Full name of agency", "Country", "Year"]

# display(companies_3a_actual_allcaps)

agencies_merged = pd.merge(agencies_3b_ideal, agencies_3b_actual_allcaps, on=['Full name of agency', 'Country', 'Year'], how='left')

# agencies_merged.to_csv("data/cleaned/agencies_5_.csv", index=False)

display(agencies_merged)
agencies_merged.to_csv("data/cleaned/agencies_3b_merge_full.csv", index=False)

,Full name of agency,Total reported_x,Country,Year,Agency type,ID number (if applicable),Total reported_y,ISO Code,Start Date,End Date
0,MINISTRY OF FINANCE (CUSTOMS DEPARTMENT),1.459912e+09,Afghanistan,2018,Central goverment,Not applicable,1367722942,AFG,2017/12/21,2018/12/20
1,MINISTRY OF FINANCE (REVENUE DEPARTMENT),1.089699e+09,Afghanistan,2018,Central goverment,Not applicable,1441453501,AFG,2017/12/21,2018/12/20
2,MINISTRY OF INDUSTRY AND COMMERCE,0.000000e+00,Afghanistan,2018,Central goverment,Not applicable,NaN,AFG,2017/12/21,2018/12/20
3,MINISTRY OF MINES AND PETROLEUM (REVENUE DEPAR...,2.285251e+09,Afghanistan,2018,Central goverment,Not applicable,1478934310.56,AFG,2017/12/21,2018/12/20
4,NATIONAL ENVIRONMENTAL PROTECTION AGENCY,4.700000e+04,Afghanistan,2018,Central goverment,Not applicable,34000,AFG,2017/12/21,2018/12/20
...,...,...,...,...,...,...,...,...,...,...
541,MINISTRY OF LANDS,9.827895e+05,Zambia,2019,Central goverment,NaN,982789.47,ZMB,2019/01/01,2019/12/31
542,MINISTRY OF MINES AND MINERALS DEVELOPMENT,4.128300e+07,Zambia,2019,Central goverment,NaN,41282998.4,ZMB,2019/01/01,2019/12/31
543,MINISTRY OF MINES AND MINERALS DEVELOPMENT - P...,3.051347e+06,Zambia,2019,Central goverment,NaN,3051347.36,ZMB,2019/01/01,2019/12/31
544,ZAMBIA CONSOLIDATED COPPER MINES – INVESTMENT ...,0.000000e+00,Zambia,2019,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
# Group ideal by Country and Year
ideal_groups = agencies_3a_ideal.groupby(['Country', 'Year'])

# Initialize an empty list to store merged DataFrames
agencies_merged_list = []

# agencies_3a_actual_allcaps['Country'] 

# Iterate over each group and merge with actual
for (country, year), group_ideal in ideal_groups:
    # display(group_ideal)
    group_actual = agencies_3a_actual_allcaps[(agencies_3a_actual_allcaps['Country'] == country) & (agencies_3a_actual_allcaps['Year'] == year)]
    # display(group_actual)
    agencies_merged_tolist = pd.merge(group_ideal, group_actual, on=['Full name of agency', 'Country', 'Year'], how='left')
    # display(agencies_merged_tolist)
    agencies_merged_list.append(agencies_merged_tolist)
    # display(agencies_merged_list)

# Concatenate the list of merged DataFrames into a single DataFrame
agencies_final_merged = pd.concat(agencies_merged_list)

display(agencies_final_merged)
agencies_final_merged.to_csv("data/cleaned/agencies_3b_merge_per_report.csv", index=False)

NameError: name 'agencies_3a_ideal' is not defined